In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt

In [2]:
train_path="/content/drive/MyDrive/Datasets/"

In [6]:
x_train=[]

for folder in os.listdir(train_path):
    sub_path=train_path+"/"+folder
    for img in os.listdir(sub_path):
      try:
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x_train.append(img_arr)
      except Exception as e:  # in the interest in keeping the output clean...
        pass

In [8]:
train_x=np.array(x_train)
train_x=train_x/255.0

In [9]:
train_datagen = ImageDataGenerator(rescale = 1./255)

In [10]:
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'sparse')

Found 908 images belonging to 9 classes.


In [11]:
train_y=training_set.classes

In [13]:
training_set.class_indices
train_y.shape

(908,)

In [16]:
IMAGE_SIZE = [224,224]
vgg = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

80150528/80134624 [==============================] - 1s 0us/step


In [17]:
for layer in vgg.layers:
    layer.trainable = False

In [18]:
x = Flatten()(vgg.output)

In [25]:
#adding output layer.Softmax classifier is used as it is multi-class classification
prediction = Dense(9, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [26]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [27]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer="adam",
  metrics=['accuracy']
)

In [28]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=5)
#Early stopping to avoid overfitting of model

In [29]:
# fit the model
history = model.fit(
  train_x,
  train_y,
  epochs=10,
  callbacks=[early_stop],
  batch_size=32,shuffle=True)

Epoch 1/10
29/29 [==============================] - 567s 20s/step - loss: 2.3282 - accuracy: 0.4240
Epoch 2/10
29/29 [==============================] - 569s 20s/step - loss: 0.8734 - accuracy: 0.6850
Epoch 3/10
29/29 [==============================] - 564s 19s/step - loss: 0.5141 - accuracy: 0.8117
Epoch 4/10
29/29 [==============================] - 563s 19s/step - loss: 0.4888 - accuracy: 0.8502
Epoch 5/10
29/29 [==============================] - 562s 19s/step - loss: 0.3817 - accuracy: 0.8722
Epoch 6/10
29/29 [==============================] - 560s 19s/step - loss: 0.3479 - accuracy: 0.8998
Epoch 7/10
29/29 [==============================] - 564s 19s/step - loss: 0.3302 - accuracy: 0.8998
Epoch 8/10
29/29 [==============================] - 561s 19s/step - loss: 0.3012 - accuracy: 0.9108
Epoch 9/10
29/29 [==============================] - 561s 19s/step - loss: 0.3340 - accuracy: 0.8976
Epoch 10/10
29/29 [==============================] - 561s 19s/step - loss: 0.3057 - accuracy: 0.9031